In [2]:
import os
import torch
from torch import nn
from tqdm import tqdm
import numpy as np
import pandas as pd
from model import perturbation,CNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
import matplotlib.pyplot as plt
from random import sample

path = r'D:\20240412\wei\all.csv'
label_path = r'D:\20240412\alllab.csv'
wl_path = r'D:\20240412\wl.csv'
save_path = r'D:\20240412\wei\cnn'

# super iter
num_feat = 10
random_seed = 2024  
batch_size = 100
lr = 5e-2
num_epochs = 800
step_epoch = 1
new_feat_size = 300
mean_times = 50
new_feat = []
new_lab = []

######### read raw data ####################
raw_feat = pd.read_csv(path,header=None).values.astype(np.float32)[:,12195-4096:12195]
raw_label = pd.read_csv(label_path,header=None).values.astype(np.float32)
raw_wl = pd.read_csv(wl_path,header=None).values.astype(np.float32)[12201-4096:12201]

######### make new feat ####################
unique_lab = np.unique(raw_label)
for labs in unique_lab:
    index = np.isin(raw_label,labs).reshape(-1)
    unique_feat = raw_feat[index,:]
    for _ in range(new_feat_size):
        new_feat.append(np.mean(unique_feat[sample(range(0,len(unique_feat[:,0]-1)),mean_times),:],axis = 0))
        new_lab.append(labs)

new_feat = np.array(new_feat)
new_lab = np.array(new_lab).reshape(-1,1)

# scaler = MinMaxScaler()
# new_feat = scaler.fit_transform(new_feat.T).T




In [2]:
# 取测试集
unique_lab = np.unique(new_lab)

select_lab = unique_lab[6]

In [3]:


norm_feat = []


test_index = np.isin(new_lab,select_lab).reshape(-1)#-600 500 -1 50 

# # normalization
# feat_scaler = MinMaxScaler()
# lab_scaler = MinMaxScaler()

# for spec in range(len(new_feat[:,0])):
#     norm_feat.append(prep.Normalization(new_feat[spec,:]) )
# norm_feat = np.array(norm_feat) * 1000
norm_feat = new_feat*1000
# norm_lab = lab_scaler.fit_transform(raw_label)

#buyong scaler

test_feat = norm_feat[test_index,:]
test_label = new_lab[test_index]
train_feat = norm_feat[~test_index,:]
train_label = new_lab[~test_index]

# random seeding

np.random.seed(random_seed)  
random_index = [i for i in range(len(train_label))]
np.random.shuffle(random_index)

train_feat = train_feat[random_index,:]
train_label = train_label[random_index]


# # import select
index_feat = pd.read_csv(r'D:\20240412\select_wl.csv',header=None).values[:,1].astype(int)
index_feat = index_feat + 2048
'CNN不用index'
# train_feat = train_feat[:, index_feat]
# test_feat = test_feat[:, index_feat]

select_wl = raw_wl[index_feat]

# split train and test
# x_train,x_test,y_train,y_test = train_test_split(
#     norm_feat,norm_lab, test_size=0.2,shuffle = False)

x_train,x_test,y_train,y_test = train_feat,test_feat,train_label,test_label



train_data = torch.tensor(x_train, dtype=torch.float32)
train_label = torch.tensor(y_train, dtype=torch.float32)
train_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(train_data, train_label), batch_size=batch_size, shuffle=True)

test_data = torch.tensor(x_test, dtype=torch.float32)
test_label = torch.tensor(y_test, dtype=torch.float32)
test_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(test_data, test_label), batch_size=batch_size, shuffle=True)
'''remember to set shuffle to False'''


# parameter define

if torch.cuda.is_available():
    device = 'cuda'
    print("Using GPU !")
else:
    device = 'cpu'

logdir = os.getcwd()
device = 'cpu'
path = r'D:\20240412\wei\cnn' + '\\' + str(select_lab) + 'best_net.pth'
model = torch.load(path)
perturb = perturbation(4096)
criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
perturb_optimizer = torch.optim.Adam(perturb.parameters(), lr=1e-1)

Using GPU !


In [4]:
# variable define
all_sigma = []
y_train1 = []
y_test1 = []
y_train2 = []
y_test2 = []
## 模型训练
model.to(device)
model.eval()

# with torch.no_grad():
#     sum_train_r2 = 0
#     sum_test_r2 = 0
#     for x, y in train_dataloader:
#         x = x.to(device)
#         y = y.to(device)
#         preds = model(x)

#         # 评估指标
#         # y_denormalized = lab_scaler.inverse_transform(y.cpu())
#         # preds_denormalized = lab_scaler.inverse_transform(preds.cpu())
#         y_denormalized = y
#         preds_denormalized = preds
#         sum_train_r2 = sum_train_r2 + r2_score(y_denormalized,preds_denormalized)
#         y_train1.append(preds_denormalized.numpy().reshape(-1,1))
#         y_train2.append(y_denormalized.numpy().reshape(-1,1))
#         # 计算损失
#     for x_t, y_t in test_dataloader:
#         x_t = x_t.to(device)
#         y_t = y_t.to(device)
#         preds = model(x_t)

#         # 评估指标
#         # y_denormalized = lab_scaler.inverse_transform(y_t.cpu())
#         # preds_denormalized = lab_scaler.inverse_transform(preds.cpu())
#         y_denormalized = y_t
#         preds_denormalized = preds

#         sum_test_r2 = sum_test_r2 + r2_score(y_denormalized,preds_denormalized)
#         y_test1.append(preds_denormalized.numpy().reshape(-1,1))
#         y_test2.append(y_denormalized.numpy().reshape(-1,1))
        
#     sum_train_r2 = sum_train_r2 / len(train_dataloader) #数据存到cpu= 直接用.item()也行
#     sum_test_r2 = sum_test_r2 / len(test_dataloader)

# print(sum_train_r2)
# print(sum_test_r2)
# np.savetxt(save_path + '//' + 'Y_predict.csv',
#            np.hstack((np.vstack((np.array(y_train1).reshape(-1,1),np.array(y_test1).reshape(-1,1))),
#                       np.vstack((np.array(y_train2).reshape(-1,1),np.array(y_test2).reshape(-1,1))))),
#            delimiter=',',fmt='%s')

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=8192, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=1, bias=True)
  )
)

In [5]:
model.cpu()
perturb.cpu()
another_epoch = int(num_epochs/40)

train_dataloader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(train_data, train_label), batch_size=150, shuffle=False)

for epochs in range(another_epoch):
    loop = tqdm(train_dataloader, total=len(test_dataloader))
    for x, y in loop:
        x = x.cpu()
        pp,pm,sigma = perturb(x)
        sigma = sigma.cpu().detach()#记得换个位置存
        p = model(x)
        pp = model(pp)
        pm = model(pm)
        pp = torch.abs(pp - p)
        pm = torch.abs(pm - p)
        const = torch.full([pp.size()[0],1],torch.max(p).item())
        const_0 = torch.full([pp.size()[0],1],torch.min(p).item())

        l1 = criterion(pp,const_0)
        l2 = criterion(pm,const)
        l3 = criterion(sigma,torch.zeros(4096))

        loss = l1 + l2 + l3


        perturb_optimizer.zero_grad()
        loss.backward()

        perturb_optimizer.step()
        
        loop.set_description('epoch: [%d/%d]' % (epochs, another_epoch))

    if epochs % step_epoch == 0: # 每step_epoch次epoch存一次数据
        save_sigma = sigma.clone().numpy()
        all_sigma.append(save_sigma)

epoch: [0/20]: : 24it [00:40,  1.68s/it]                    
epoch: [1/20]: : 24it [00:41,  1.73s/it]                    
epoch: [2/20]: : 24it [00:41,  1.74s/it]                    
epoch: [3/20]: : 24it [00:42,  1.75s/it]                    
epoch: [4/20]: : 24it [00:41,  1.73s/it]                    
epoch: [5/20]: : 24it [00:41,  1.74s/it]                    
epoch: [6/20]: : 24it [00:41,  1.73s/it]                    
epoch: [7/20]: : 24it [00:41,  1.74s/it]                    
epoch: [8/20]: : 24it [00:41,  1.75s/it]                    
epoch: [9/20]: : 24it [00:42,  1.75s/it]                    
epoch: [10/20]: : 24it [00:42,  1.75s/it]                    
epoch: [11/20]: : 24it [00:41,  1.74s/it]                    
epoch: [12/20]: : 24it [00:41,  1.74s/it]                    
epoch: [13/20]: : 24it [00:41,  1.75s/it]                    
epoch: [14/20]: : 24it [00:41,  1.75s/it]                    
epoch: [15/20]: : 24it [00:41,  1.73s/it]                    
epoch: [16/20]: : 

In [6]:
# scaler = MinMaxScaler()
# save_sigma = scaler.fit_transform(save_sigma.reshape(-1,1))

In [7]:
a = np.array(all_sigma)

In [8]:
save_sigma

array([0.25589982, 0.56894714, 0.8381044 , ..., 0.4142936 , 0.2013672 ,
       0.48210844], dtype=float32)

In [9]:
raw_wl[np.argmax(save_sigma)]

array([676.69], dtype=float32)

In [10]:
np.savetxt(save_path + '//' + 'sigma11.csv',np.vstack((raw_wl.T,a)).T,delimiter=',',fmt='%s')

In [11]:
aa = np.std(a,axis=0)
scaler = MinMaxScaler()
aa = scaler.fit_transform(aa.reshape(-1,1))

In [12]:
np.savetxt(save_path + '//' + 'aa.csv',aa,delimiter=',',fmt='%s')

In [13]:
np.savetxt(save_path + '//' + 'sigma.csv',all_sigma[-1],delimiter=',',fmt='%s')

In [14]:
np.savetxt(save_path + '//' + 'save_sigma.csv',np.hstack((raw_wl,save_sigma)),delimiter=',',fmt='%s')

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
plt.figure(1)
x = np.linspace(0,100,len(all_sigma[0]))
plt.plot(x,all_sigma[0])
# plt.plot(x,all_sigma[-1],c = 'r')